In [1]:
""" combine query predictions and output as a result json file """
import os
from utils import execute_all

path1 = "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_0.json"
path2 = "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_1.json"

# combine two json files
import json
with open(path1) as f:
    data1 = json.load(f)
with open(path2) as f:
    data2 = json.load(f)

data1.update(data2)

DB_PATH = os.path.join('data', 'mimic_iv', 'mimic_iv.sqlite')
predictions = execute_all(data1, db_path=DB_PATH, tag='pred')

with open("./RESULTS/baselinev3-flan-t5-large_fold8/predictions.json", "w") as f:
    json.dump(predictions, f, indent=4)

### Hard Vote Ensemble

In [2]:
import json

PATHS = [  
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold2/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold3/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold4/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold5/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold6/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold7/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold8/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold9/predictions.json",
]

In [3]:
import pandas as pd

df = pd.DataFrame(columns=["id"])
for idx, json_path in enumerate(PATHS):
  column_name = f"answer_{idx}"
  df[column_name] = None
  with open(json_path, "r") as f:
    data = json.load(f)
    if idx == 0: # initialize column
      df["id"] = list(data.keys())
    for key in data: # append data
      df.loc[df["id"] == key, column_name] = data[key]

df.head(10)

,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
0,282f008dd8dfb8f4a1dd6999,[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']]
1,7f07c59357750ac2b84e5221,"[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']...","[['blood culture'], ['bronchoalveolar lavage']..."
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
3,dfc058cc8e8d30a29c0177b9,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]
4,647bc08d18cac6dbda791c42,[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']]
5,5436413908d5e902536630e8,[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
7,3fc7c5e0ee947f0f155f81ae,[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']],[['2100-01-27 15:00:00']]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null


In [4]:
# find rows with different answers
diff_row_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if len(set(answers)) > 1:
    diff_row_idxs.append(idx)

df_diff = df.loc[diff_row_idxs]
print(f"Number of different rows: {len(df_diff)}")
df_diff.head(10)

Number of different rows: 415


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
11,fa7797af0bc8368e7b76a129,[['None']],[['None']],[['None']],[['-6.0']],[['-6.0']],[['None']],[['-6.0']],[['None']],[['None']],[['None']]
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
21,104c93f89e19274578bafb8d,[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-11 13:43:00']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred


In [5]:
# iterrate over rows and check error_pred
error_pred_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers:
    error_pred_idxs.append(idx)

df_error_pred = df.loc[error_pred_idxs]
print(f"Number of error_pred rows: {len(df_error_pred)}")
df_error_pred.head(10)

Number of error_pred rows: 321


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred
23,271fe9a047e24d8cef13cf91,error_pred,error_pred,[],error_pred,null,[],error_pred,[],[],error_pred
24,297a1435453b2870054eb81d,null,error_pred,error_pred,[['None']],error_pred,null,null,null,null,error_pred


In [6]:
# check for rows that either have error_pred or different answers
error_pred_diff_idxs = list(set(diff_row_idxs + error_pred_idxs))
df_error_pred_diff = df.loc[error_pred_diff_idxs]
print(f"Number of error_pred or different rows: {len(df_error_pred_diff)}")
df_error_pred_diff.head(10)

Number of error_pred or different rows: 419


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
11,fa7797af0bc8368e7b76a129,[['None']],[['None']],[['None']],[['-6.0']],[['-6.0']],[['None']],[['-6.0']],[['None']],[['None']],[['None']]
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
21,104c93f89e19274578bafb8d,[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-11 13:43:00']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred


In [8]:
df_error_pred_diff.tail(10)

,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
1135,0d6e154c99ea7aa10dc0ac5c,[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['None']],[['50.0']],[['50.0']],[['50.0']]
1137,207422a6435e8494054f044b,[['0.0']],error_pred,error_pred,[['0.0']],[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']],[['0.0']]
1139,7ed6ef4b97a89c40124bd85f,error_pred,[['0.0']],[['0.0']],error_pred,error_pred,[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']]
1140,ec1ed057dd7ec2a55f903e6e,[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]
1144,debd4fb35717b39a57ee2bec,[['7.18']],[],[['7.18']],[['7.18']],[],[['7.18']],[],[],[['7.18']],[['7.18']]
1153,6985b7163e6c8b9db7102745,[['0.0']],error_pred,error_pred,[['9.0']],[['9.0']],[['9.0']],[['9.0']],[['0.0']],[['0.0']],[['0.0']]
1158,b79b244dc972c7cd9869e514,null,null,null,[],null,null,null,null,null,null
1159,9df13449259f96705f0f7f56,error_pred,error_pred,error_pred,[['None']],error_pred,[['None']],error_pred,error_pred,error_pred,error_pred
1161,8101d176c45f4e285b9e9540,error_pred,[['1.0']],[['0.0']],error_pred,error_pred,[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
1164,e2740d72774b525d272c8e62,error_pred,error_pred,error_pred,[],[['0.0']],error_pred,error_pred,null,error_pred,[]


In [7]:
from collections import Counter

# write df as new json file
ensembled_dict = {}
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers: # pass the difficult question
    ensembled_dict[row["id"]] = "null"
    continue
  else:
    # answer only when the frequency is more than half of the all answers
    answer_counter = Counter(answers) # calculate answers and its frequency
    max_freq = max(answer_counter.values())
    max_freq_answer = [key for key, value in answer_counter.items() if value == max_freq]
    if max_freq > len(answers) / 2:
      ensembled_dict[row["id"]] = max_freq_answer[0]
    else:
      ensembled_dict[row["id"]] = "null"

In [9]:
import json

%cd ./RESULTS/ENSEMBLE
path = "./test_ensemble_v4_ten.json"
zip_path = path.replace(".json", ".zip")
with open(path, "w") as f:
    json.dump(ensembled_dict, f, indent=4)

!zip $zip_path $path

/ext_hdd/yjahn/joint-learning/RESULTS/ENSEMBLE
  adding: test_ensemble_v4_ten.json (deflated 73%)


/home/yjahn/miniconda3/envs/ehr/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
